# 🎰 JetX Predictor - Full Model Training (N-BEATS + TCN)

Bu notebook Google Colab'da JetX tahmin modelini eğitir.

**Eğitim Süreci:**
1. Kütüphaneleri yükle
2. GitHub'dan projeyi klonla  
3. SQLite veritabanından veri yükle (7000 değer)
4. Özellik çıkarma (Feature Engineering - 100+ özellik)
5. N-BEATS + TCN modelini oluştur
6. Modeli eğit (~45-60 dakika GPU ile)
7. Model dosyalarını otomatik indir

**Tahmini Süre:** ~1 saat (GPU ile)

---

## 📦 Adım 1: Kütüphaneleri Yükle

In [ ]:
%%capture
# Gerekli kütüphaneleri sessizce yükle
!pip install -q tensorflow scikit-learn pandas numpy scipy joblib matplotlib seaborn

In [ ]:
import sys
import os
import sqlite3
import numpy as np
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score, classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

print(f"✅ TensorFlow versiyon: {tf.__version__}")
print(f"✅ GPU kullanılabilir mi: {len(tf.config.list_physical_devices('GPU')) > 0}")

## 🔄 Adım 2: GitHub'dan Projeyi Klonla

In [ ]:
# Eğer zaten klonlanmışsa tekrar klonlama
if not os.path.exists('jetxpredictor'):
    !git clone https://github.com/onndd/jetxpredictor.git
    print("✅ Repo klonlandı")
else:
    print("✅ Repo zaten mevcut")

# Proje dizinine geç
%cd jetxpredictor

# Python path'e ekle
sys.path.append('/content/jetxpredictor')

print("✅ Proje yüklendi")

## 📚 Adım 3: Kategori Tanımlarını Yükle

In [ ]:
from category_definitions import CategoryDefinitions, FeatureEngineering, SEQUENCE_LENGTHS

print("✅ CategoryDefinitions yüklendi")
print(f"✅ Kritik eşik: {CategoryDefinitions.CRITICAL_THRESHOLD}x")
print(f"✅ Sequence uzunlukları: {SEQUENCE_LENGTHS}")

## 📊 Adım 4: Veritabanından Veri Yükle

In [ ]:
# SQLite veritabanını kontrol et
db_path = 'jetx_data.db'

if not os.path.exists(db_path):
    print(f'❌ HATA: {db_path} bulunamadı!')
    print('Lütfen jetx_data.db dosyasını Colab\'a yükleyin (Files sekmesinden).')
    raise FileNotFoundError(f'{db_path} bulunamadı')

# Veritabanından veri oku
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Tablo yapısını kontrol et
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print(f'Tablolar: {tables}')

# Veri yükle
try:
    data = pd.read_sql_query("SELECT value FROM jetx_data ORDER BY id", conn)
except:
    data = pd.read_sql_query("SELECT actual_value as value FROM predictions WHERE actual_value IS NOT NULL ORDER BY id", conn)

conn.close()

# Veriyi numpy array'e çevir
all_values = data['value'].values

print(f'\n✅ Toplam {len(all_values)} veri yüklendi')
print(f'Veri aralığı: {all_values.min():.2f}x - {all_values.max():.2f}x')
print(f'Ortalama: {all_values.mean():.2f}x')
print(f'Medyan: {np.median(all_values):.2f}x')
print(f'1.5x üstü oran: {(all_values >= 1.5).sum() / len(all_values) * 100:.1f}%')

## 🔧 Adım 5: Feature Engineering - Özellik Çıkarma

Her değer için 100+ özellik çıkarıyoruz (sliding window yöntemi ile)

In [ ]:
window_size = 500  # Minimum geçmiş veri
prediction_horizon = 1

X_features_list = []
X_seq_50_list = []
X_seq_200_list = []
X_seq_500_list = []
y_regression_list = []
y_classification_list = []

print('Özellikler çıkarılıyor...')
for i in range(window_size, len(all_values) - prediction_horizon):
    history = all_values[:i].tolist()
    target_value = all_values[i]
    
    # Özellikleri çıkar
    features = FeatureEngineering.extract_all_features(history)
    feature_values = np.array(list(features.values()))
    
    # Sequence'leri oluştur
    seq_50 = all_values[i-50:i]
    seq_200 = all_values[i-200:i]
    seq_500 = all_values[i-500:i]
    
    # Hedef değerler
    y_reg = target_value
    y_class = CategoryDefinitions.get_category_numeric(target_value)
    y_class_onehot = np.zeros(3)
    y_class_onehot[y_class] = 1
    
    X_features_list.append(feature_values)
    X_seq_50_list.append(seq_50)
    X_seq_200_list.append(seq_200)
    X_seq_500_list.append(seq_500)
    y_regression_list.append(y_reg)
    y_classification_list.append(y_class_onehot)
    
    if (i - window_size + 1) % 500 == 0:
        print(f'İşlenen: {i - window_size + 1} / {len(all_values) - window_size - prediction_horizon}')

# Numpy array'lere çevir
X_features = np.array(X_features_list)
X_seq_50 = np.array(X_seq_50_list).reshape(-1, 50, 1)
X_seq_200 = np.array(X_seq_200_list).reshape(-1, 200, 1)
X_seq_500 = np.array(X_seq_500_list).reshape(-1, 500, 1)
y_regression = np.array(y_regression_list)
y_classification = np.array(y_classification_list)

print(f'\n✅ Özellik çıkarma tamamlandı')
print(f'X_features shape: {X_features.shape}')
print(f'X_seq_50 shape: {X_seq_50.shape}')
print(f'X_seq_200 shape: {X_seq_200.shape}')
print(f'X_seq_500 shape: {X_seq_500.shape}')

## 📐 Adım 6: Normalizasyon ve Train/Test Split

In [ ]:
# Features için StandardScaler
scaler = StandardScaler()
X_features_scaled = scaler.fit_transform(X_features)

# Sequence'ler için log transform
def normalize_sequence(seq):
    return np.log10(seq + 1e-8)

X_seq_50_norm = normalize_sequence(X_seq_50)
X_seq_200_norm = normalize_sequence(X_seq_200)
X_seq_500_norm = normalize_sequence(X_seq_500)

# Train/Test split (%80/%20)
test_size = 0.2
indices = np.arange(len(X_features_scaled))
train_idx, test_idx = train_test_split(indices, test_size=test_size, shuffle=False)

# Train data
X_features_train = X_features_scaled[train_idx]
X_seq_50_train = X_seq_50_norm[train_idx]
X_seq_200_train = X_seq_200_norm[train_idx]
X_seq_500_train = X_seq_500_norm[train_idx]
y_reg_train = y_regression[train_idx]
y_class_train = y_classification[train_idx]

# Test data
X_features_test = X_features_scaled[test_idx]
X_seq_50_test = X_seq_50_norm[test_idx]
X_seq_200_test = X_seq_200_norm[test_idx]
X_seq_500_test = X_seq_500_norm[test_idx]
y_reg_test = y_regression[test_idx]
y_class_test = y_classification[test_idx]

print(f'✅ Veri normalize edildi ve bölündü')
print(f'Train samples: {len(X_features_train)}')
print(f'Test samples: {len(X_features_test)}')
print(f'Feature boyutu: {X_features_train.shape[1]}')

## 🏗️ Adım 7: N-BEATS + TCN Model Mimarisi

In [ ]:
n_features = X_features_train.shape[1]

# INPUT LAYERS
input_features = layers.Input(shape=(n_features,), name='input_features')
input_seq_50 = layers.Input(shape=(50, 1), name='input_seq_50')
input_seq_200 = layers.Input(shape=(200, 1), name='input_seq_200')
input_seq_500 = layers.Input(shape=(500, 1), name='input_seq_500')

# N-BEATS BLOKLARI
def create_nbeats_block(input_layer, units, num_blocks, name_prefix):
    x = input_layer
    for i in range(num_blocks):
        x = layers.Dense(units, activation='relu', name=f'{name_prefix}_block{i}_dense1')(x)
        x = layers.Dropout(0.2)(x)
        x = layers.Dense(units, activation='relu', name=f'{name_prefix}_block{i}_dense2')(x)
        x = layers.Dropout(0.2)(x)
    return x

nbeats_short = layers.Flatten()(input_seq_50)
nbeats_short = create_nbeats_block(nbeats_short, 64, 3, 'nbeats_short')
nbeats_short = layers.Dense(64, activation='relu', name='nbeats_short_output')(nbeats_short)

nbeats_medium = layers.Flatten()(input_seq_200)
nbeats_medium = create_nbeats_block(nbeats_medium, 128, 4, 'nbeats_medium')
nbeats_medium = layers.Dense(128, activation='relu', name='nbeats_medium_output')(nbeats_medium)

nbeats_long = layers.Flatten()(input_seq_500)
nbeats_long = create_nbeats_block(nbeats_long, 256, 5, 'nbeats_long')
nbeats_long = layers.Dense(256, activation='relu', name='nbeats_long_output')(nbeats_long)

nbeats_combined = layers.Concatenate(name='nbeats_combined')([nbeats_short, nbeats_medium, nbeats_long])

# TCN MODÜLÜ
def create_tcn_block(input_layer, filters, kernel_size, dilation_rate, name_prefix):
    conv = layers.Conv1D(
        filters=filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding='causal',
        activation='relu',
        name=f'{name_prefix}_conv'
    )(input_layer)
    conv = layers.Dropout(0.2)(conv)
    
    if input_layer.shape[-1] != filters:
        residual = layers.Conv1D(filters, 1, padding='same', name=f'{name_prefix}_residual')(input_layer)
    else:
        residual = input_layer
    
    return layers.Add(name=f'{name_prefix}_add')([conv, residual])

tcn = input_seq_500
tcn = create_tcn_block(tcn, 64, 3, 1, 'tcn_block1')
tcn = create_tcn_block(tcn, 64, 3, 2, 'tcn_block2')
tcn = create_tcn_block(tcn, 128, 3, 4, 'tcn_block3')
tcn = create_tcn_block(tcn, 128, 3, 8, 'tcn_block4')
tcn = create_tcn_block(tcn, 256, 3, 16, 'tcn_block5')
tcn = create_tcn_block(tcn, 256, 3, 32, 'tcn_block6')

tcn_output = layers.GlobalAveragePooling1D(name='tcn_global_pool')(tcn)
tcn_output = layers.Dense(512, activation='relu', name='tcn_output')(tcn_output)

# ENSEMBLE FUSION
combined = layers.Concatenate(name='ensemble_fusion')([input_features, nbeats_combined, tcn_output])

fusion = layers.Dense(512, activation='relu', name='fusion_dense1')(combined)
fusion = layers.Dropout(0.3)(fusion)
fusion = layers.Dense(256, activation='relu', name='fusion_dense2')(fusion)
fusion = layers.Dropout(0.3)(fusion)
fusion = layers.Dense(128, activation='relu', name='fusion_dense3')(fusion)

# MULTI-OUTPUT HEADS
regression_head = layers.Dense(64, activation='relu', name='regression_head')(fusion)
output_regression = layers.Dense(1, activation='linear', name='output_regression')(regression_head)

classification_head = layers.Dense(64, activation='relu', name='classification_head')(fusion)
output_classification = layers.Dense(3, activation='softmax', name='output_classification')(classification_head)

confidence_head = layers.Dense(32, activation='relu', name='confidence_head')(fusion)
output_confidence = layers.Dense(1, activation='sigmoid', name='output_confidence')(confidence_head)

risk_head = layers.Dense(32, activation='relu', name='risk_head')(fusion)
output_pattern_risk = layers.Dense(1, activation='sigmoid', name='output_pattern_risk')(risk_head)

# MODEL
model = models.Model(
    inputs=[input_features, input_seq_50, input_seq_200, input_seq_500],
    outputs=[output_regression, output_classification, output_confidence, output_pattern_risk],
    name='JetX_NBEATS_TCN_Model'
)

print('✅ Model oluşturuldu')
model.summary()

## ⚙️ Adım 8: Model Compile

In [ ]:
# Custom loss: 1.5x threshold için özel ağırlık
def threshold_weighted_mae(y_true, y_pred):
    mae = tf.abs(y_true - y_pred)
    threshold_weight = tf.where(
        tf.logical_and(y_true >= 1.4, y_true <= 1.6),
        2.0,
        1.0
    )
    weighted_mae = mae * threshold_weight
    return tf.reduce_mean(weighted_mae)

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss={
        'output_regression': threshold_weighted_mae,
        'output_classification': 'categorical_crossentropy',
        'output_confidence': 'binary_crossentropy',
        'output_pattern_risk': 'binary_crossentropy'
    },
    loss_weights={
        'output_regression': 0.4,
        'output_classification': 0.3,
        'output_confidence': 0.2,
        'output_pattern_risk': 0.1
    },
    metrics={
        'output_regression': ['mae', 'mse'],
        'output_classification': ['accuracy'],
        'output_confidence': ['accuracy'],
        'output_pattern_risk': ['accuracy']
    }
)

print('✅ Model compile edildi')

## 🎛️ Adım 9: Callbacks Hazırlama

In [ ]:
# Hedef değerleri hazırla
y_confidence_train = np.where(
    (y_reg_train >= 1.45) & (y_reg_train <= 1.55),
    0.3,
    0.7
)

y_risk_train = np.where(y_reg_train < 1.5, 0.8, 0.2)

y_confidence_test = np.where(
    (y_reg_test >= 1.45) & (y_reg_test <= 1.55),
    0.3,
    0.7
)

y_risk_test = np.where(y_reg_test < 1.5, 0.8, 0.2)

# Callbacks
checkpoint = callbacks.ModelCheckpoint(
    'jetx_model_best.h5',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=7,
    min_lr=1e-6,
    verbose=1
)

print('✅ Callbacks hazırlandı')

## 🚀 Adım 10: Model Eğitimi - BAŞLAT!

⚠️ **Bu adım ~45-60 dakika sürebilir (GPU ile)**

In [ ]:
print('🚀 Model eğitimi başlıyor...')
print(f'Epochs: 100 (early stopping ile)')
print(f'Batch size: 32')
print(f'Validation split: 20%')
print('\n' + '='*50 + '\n')

history = model.fit(
    [X_features_train, X_seq_50_train, X_seq_200_train, X_seq_500_train],
    {
        'output_regression': y_reg_train,
        'output_classification': y_class_train,
        'output_confidence': y_confidence_train,
        'output_pattern_risk': y_risk_train
    },
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[checkpoint, early_stopping, reduce_lr],
    verbose=1
)

print('\n' + '='*50)
print('✅ Eğitim tamamlandı!')

## 📈 Adım 11: Training History Görselleştirme

In [ ]:
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['output_regression_mae'], label='Train MAE')
plt.plot(history.history['val_output_regression_mae'], label='Val MAE')
plt.title('Regression MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

print(f'En iyi validation loss: {min(history.history["val_loss"]):.4f}')
print(f'En iyi MAE: {min(history.history["val_output_regression_mae"]):.4f}')

## 📊 Adım 12: Test Seti Değerlendirme

In [ ]:
print('Test seti değerlendiriliyor...')

predictions = model.predict(
    [X_features_test, X_seq_50_test, X_seq_200_test, X_seq_500_test],
    verbose=0
)

pred_regression = predictions[0].flatten()
pred_classification = predictions[1]

# Metrikleri hesapla
mae = mean_absolute_error(y_reg_test, pred_regression)
rmse = np.sqrt(mean_squared_error(y_reg_test, pred_regression))

# 1.5x threshold accuracy
y_threshold_true = (y_reg_test >= 1.5).astype(int)
y_threshold_pred = (pred_regression >= 1.5).astype(int)
threshold_accuracy = accuracy_score(y_threshold_true, y_threshold_pred)

# Classification accuracy
y_class_true = np.argmax(y_class_test, axis=1)
y_class_pred = np.argmax(pred_classification, axis=1)
class_accuracy = accuracy_score(y_class_true, y_class_pred)

print('\n' + '='*50)
print('TEST SETİ SONUÇLARI')
print('='*50)
print(f'\n📊 REGRESSION METRİKLERİ:')
print(f'  MAE: {mae:.4f}')
print(f'  RMSE: {rmse:.4f}')
print(f'\n🎯 1.5x EŞİK DOĞRULUĞU (EN ÖNEMLİ):')
print(f'  Accuracy: {threshold_accuracy*100:.2f}%')
print(f'  Hedef: >75%')
if threshold_accuracy >= 0.75:
    print('  ✅ HEDEF BAŞARILI!')
else:
    print('  ⚠️ Hedefin altında')
print(f'\n📁 KATEGORİ DOĞRULUĞU:')
print(f'  Accuracy: {class_accuracy*100:.2f}%')
print(f'  Hedef: >60%')
if class_accuracy >= 0.60:
    print('  ✅ HEDEF BAŞARILI!')
else:
    print('  ⚠️ Hedefin altında')

print(f'\n📋 DETAYLI KATEGORİ RAPORU:')
print(classification_report(
    y_class_true,
    y_class_pred,
    target_names=['Kayıp (<1.5x)', 'Güvenli (1.5-10x)', 'Yüksek (>10x)']
))

## 💾 Adım 13: Model Kaydetme

In [ ]:
# Model kaydet
model.save('jetx_model.h5')
print('✅ Model kaydedildi: jetx_model.h5')

# Scaler kaydet
joblib.dump(scaler, 'scaler.pkl')
print('✅ Scaler kaydedildi: scaler.pkl')

# Model bilgileri
import json
model_info = {
    'n_features': int(n_features),
    'threshold_accuracy': float(threshold_accuracy),
    'class_accuracy': float(class_accuracy),
    'mae': float(mae),
    'rmse': float(rmse),
    'train_samples': len(X_features_train),
    'test_samples': len(X_features_test),
    'total_epochs': len(history.history['loss'])
}

with open('model_info.json', 'w') as f:
    json.dump(model_info, f, indent=2)

print('✅ Model bilgileri kaydedildi: model_info.json')
print(json.dumps(model_info, indent=2))

## 📥 Adım 14: Dosyaları İndirin!

**Aşağıdaki hücreyi çalıştırarak model dosyalarını bilgisayarınıza indirin.**

In [ ]:
from google.colab import files

print('Model dosyaları indiriliyor...')
print('\n⏬ Tarayıcınız indirme isteyecek, lütfen kabul edin.\n')

print('1/3 - jetx_model.h5 indiriliyor...')
files.download('jetx_model.h5')

print('2/3 - scaler.pkl indiriliyor...')
files.download('scaler.pkl')

print('3/3 - model_info.json indiriliyor...')
files.download('model_info.json')

print('\n' + '='*70)
print('✅ TÜM DOSYALAR İNDİRİLDİ!')
print('='*70)
print('\nSONRAKİ ADIMLAR:')
print('1. İndirilen dosyaları lokal projenizin models/ klasörüne kopyalayın')
print('2. Streamlit uygulamanızı çalıştırın: streamlit run app.py')
print('3. Tahmin yapmaya başlayın!')
print('\n📊 Model Performansı:')
print(f'  - 1.5x Eşik Doğruluğu: {threshold_accuracy*100:.2f}%')
print(f'  - Kategori Doğruluğu: {class_accuracy*100:.2f}%')
print(f'  - MAE: {mae:.4f}')
print('\n🎉 Başarılar!')